In [ ]:
import time
import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split, KFold

from utils import *

In [ ]:
mnist = fetch_mldata( "MNIST original" )
trainX, testX, trainY, testY = train_test_split( mnist.data, mnist.target, test_size = 10000,
                                           random_state = 123 )

trainX = trainX.reshape( (len(trainX), 28, 28, 1) )
testX  = testX.reshape(  (len(testX),  28, 28, 1) )

In [ ]:
nines = [ 0.9, 0.99, 0.999, 0.9999, 0.99999 ]
sizes = [ 16 ] #[ 32, 64, 128, 256, 512, 1024 ]
paramsList = [ { "batchSize" : n, "params" : { "alpha" : a, "momentum" : b1, "bnm" : b2 } } \
               for a in np.logspace(-4,-2, 5) for b1 in nines for b2 in nines for n in sizes ]

loVal, trHist, vlHist, bestParams = hyperparameterSearch( trainX, trainY, paramsList, 3 )

print("\n\nLowest validation loss:", loVal)
print("Best parameters found:", bestParams )

In [ ]:
fig = plt.figure( 1, (9, 7) )

plt.plot( trHist[:], label = "training" )
plt.plot( vlHist[:], label = "validation" )

plt.xlim( xmin = 0, xmax = len(trHist) - 1 )
plt.ylim( ymin = 0, ymax = max(trHist[1], vlHist[1]) + .5 )

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.legend( loc = "best" )

plt.show()

#fig.savefig( "learningBN.eps", format = 'eps', dpi = 20000, bbox_inches = 'tight' )

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape = (None, n), name = "X")
y = tf.placeholder(tf.int32, shape = (None), naame = "y")
trn = tf.placeholder_with_default( False, shape = (), name = "trn" )

loss, training, accuracy, lossSummary, init, saver = mnistCNN( X, y, trn, **(bestParams["params"]) )

with tf.Session() as sess:

    saver.restore( sess, "./best/mnist-best.ckpt" )
    testAcc = accuracy.eval( feed_dict = { X : hTeX, y : hTeY })

    print( "Accuracy on test set:", testAcc )